<a href="https://colab.research.google.com/github/Remache-Abderraheem/ChatbotCollab/blob/main/rag_video_engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import auth
auth.authenticate_user()


In [ ]:
pip install -U openai-whisper


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.6/798.6 kB 4.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 22.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using ca

In [ ]:
from google.cloud import firestore
import whisper
from google.cloud import storage
from whisper.utils import get_writer
import re

whisper_bucket = "rag-videos-engine-whisper"

def is_valid_text(text):
  # Remove all special characters
  cleaned_text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
  # Split the cleaned text into words
  words = cleaned_text.split()
  # Check if the number of words exceeds 30
  if len(words) > 30:
      return True
  else:
      return False


def download_video(blob_name) -> str:

    # Initialize GCS client
    client = storage.Client()

    # Replace 'your-bucket-name' with the name of your GCS bucket
    bucket = client.bucket('rag-videos-engine-downloads')
    try:
      blob = bucket.blob(blob_name)

      blob.download_to_filename("vid.mp4")
    except Exception as e:
          print("Error occurred:", e)



def save_file(results, format='tsv'):
    writer = get_writer(format, './')
    writer(results, f'transcribe.{format}')


def upload_file_to_bucket(bucket_name, folder_name, file_name, file_content):
    """Uploads a file to a Cloud Storage bucket."""
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(folder_name + "/" + file_name)
    blob.upload_from_string(file_content)


# Initialize Firestore client
db = firestore.Client(project="rag-videos-engine")

# Replace 'your-collection-name' with the name of your collection
collection_name = 'video-source'

doc_ref = db.collection(collection_name)
docs = doc_ref.get()

testModel = whisper.load_model("base")
processingModel = whisper.load_model("large")


for doc in docs:
  wstatus = []
  print(doc.id)
  doc_dict = doc.to_dict()
  title_array = doc_dict.get("Title")
  text = doc_dict.get("Text")
  status_array = doc_dict.get("Status")
  whisper_array = doc_dict.get("WStatus")
  if whisper_array:
    print("no more to add")
  else:
    index = 0
    for title,status in zip(title_array, status_array):
      if status == "OK":
        print(title)
        download_video(title+".mp4")
        try:
          test = testModel.transcribe("vid.mp4", task='translate')
          print(test)
        except Exception as e:
          print("Error occurred:", e)
          wstatus.append("Mute")
          index+=1
          continue  # Continue to the next iteration of the loop
        if is_valid_text(test["text"]):
            print("Valid text")
            result = processingModel.transcribe("vid.mp4", task='translate')
            types = ["txt", "vtt", "srt", "tsv", "json"]
            for ftype in types:
              save_file(result, ftype)
            folder = doc.id+"/"+str(index)
            for ftype in types:
              with open("transcribe."+ftype, 'r') as file:
                # Read the content of the file
                file_content = file.read()
              upload_file_to_bucket(whisper_bucket,folder,"transcribe."+ftype,file_content)
            wstatus.append("Transcribed")
            index+=1
        else:
            print("Invalid text")
            if is_valid_text(text):
              folder = doc.id+"/"+str(index)
              upload_file_to_bucket(whisper_bucket,folder,"transcribe.txt",text)
              wstatus.append("Empty")
              index+=1
            else:
              wstatus.append("Unuseful")
              index+=1
      else:
        wstatus.append("Error")
        index+=1
    doc_ref = db.collection(collection_name).document(doc.id)
    doc_ref.update({"WStatus": wstatus})

100%|████████████████████████████████████████| 139M/139M [00:00<00:00, 186MiB/s]
100%|██████████████████████████████████████| 2.88G/2.88G [00:20<00:00, 152MiB/s]


0KGhcjSix6j73xhzTiS0
El_fakhora_school_attack2
{'text': " May God give us blood, May God give his life to you. To make God awhile your weight is brother. may He give you all the wealth and glory in his Master's company. These qualities areators, Sayin As Awesome May God bless us May God bless us 5 people The biggest thing to have women to complete ust autograph and whistle You look at me by carefully He keeps enjoying you I wish Allah had simulations the mists, the kings of Jньjdrawi all the titans are united", 'segments': [{'id': 0, 'seek': 0, 'start': 0.0, 'end': 3.2600000000000002, 'text': ' May God give us blood,', 'tokens': [50364, 1891, 1265, 976, 505, 3390, 11, 50527], 'temperature': 1.0, 'avg_logprob': -3.8082466819069603, 'compression_ratio': 1.375, 'no_speech_prob': 0.4508900046348572}, {'id': 1, 'seek': 0, 'start': 5.82, 'end': 6.96, 'text': ' May God give his life to you.', 'tokens': [50655, 1891, 1265, 976, 702, 993, 281, 291, 13, 50712], 'temperature': 1.0, 'avg_logprob':